# FEM-BEM coupling 

In [1]:
from netgen.csg import *
from ngsolve import *
from ngsolve.webgui import Draw

In [2]:
ball = Sphere(Pnt(0,0,0), 1)
geo = CSGeometry()
geo.Add(ball)
mesh = Mesh(geo.GenerateMesh(maxh=0.2))

Draw (mesh)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2007-270-gcfd30775a', 'mesh_dim': 3, 'order2d': 1, 'order3d': 1…

In [3]:
order=1
V = H1(mesh,order=order)
Q = SurfaceL2(mesh, order=order-1)
X = V*Q
(u,lam), (v,mu) = X.TnT()

In [4]:
f = LinearForm(X)
f += v(0.7,0,0)  # point source
f.Assemble()

a = BilinearForm(grad(u)*grad(v)*dx - lam*v*ds).Assemble()
b = BilinearForm(grad(u)*grad(v)*dx+u*v*dx + lam*mu*ds).Assemble()
inv = b.mat.Inverse()

gf = GridFunction(X)
gf.vec.data = inv * f.vec
gfu, gflam = gf.components

In [5]:
Draw (gfu)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2007-270-gcfd30775a', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2…

Next, we assemble the corresponding boundary integral operators using Bempp-cl and ngbem

In [6]:
import ngbem
import bempp.api;
import numpy as np;
import scipy;
import bempp.core

# bempp.api.DEFAULT_DEVICE_INTERFACE="opencl"
bempp.api.DEFAULT_DEVICE_INTERFACE="numba"

In [7]:
class NGSOperator(BaseMatrix):
    def __init__(self, mat):
        BaseMatrix.__init__(self)
        self.mat = mat
    def IsComplex(self):
        return False
    def Height(self):
        return self.mat.shape[0]
    def Width(self):
        return self.mat.shape[1]
    def CreateRowVector(self):
        return BaseVector(self.Width())
    def CreateColVector(self):
        return BaseVector(self.Height())
    def Mult(self,x,y):
        y.FV().NumPy()[:] = self.mat * x    

NGBem implements the trace operator between (in our case) $H^1(\Omega)$ and $H^{1/2}(\partial \Omega)$. It
also provides the mapping between the NGSolve SurfaceL2 and the Bempp-cl space of piecewise constant functions.
In order to make sure that the grids and spaces match, H1_trace also returns a corresponding Bempp-cl function space. (The second parameter makes sure that we only generate the Bempp grid once).

Note that, since we are using piecewise constant functions for $Q$, bnd_trace_matrix will be the identity

In [8]:
[bem_c,trace_matrix]=ngbem.ng_to_bempp_trace(V);
[bem_dc, bnd_trace_matrix]=ngbem.ng_to_bempp_trace(Q,bem_c.grid);

got  332  surface vertices
doing order  1
error due to eval 0.0
doing order  0
error due to eval 0.0


We will need the following operators:
the single layer operator $V$,
the double layer operator $K$ and a mass matrix $M$

In [9]:
##set up the bem
bempp.api.VECTORIZATION_MODE = "novec" 
sl=bempp.api.operators.boundary.laplace.single_layer(bem_dc,bem_c,bem_dc) # ,assembler="fmm", device_interface="opencl")
dl=bempp.api.operators.boundary.laplace.double_layer(bem_c,bem_c,bem_dc)#,assembler="fmm", device_interface="opencl")
id_op=bempp.api.operators.boundary.sparse.identity(bem_dc,bem_dc,bem_c)
id_op2=bempp.api.operators.boundary.sparse.identity(bem_c,bem_c,bem_dc)

In [10]:
embu, emblam = X.embeddings

In [11]:
bnd_op1=0.5*id_op2 - dl
ngs_bnd1= NGSOperator(bnd_trace_matrix.T) @ NGSOperator(bnd_op1.weak_form()) @ NGSOperator(trace_matrix)
ngs_bnd1= emblam @ ngs_bnd1 @ embu.T

We set up the following block system:
$\begin{pmatrix}
A & M \\
\frac{1}{2} M - K & V
\end{pmatrix} 
\begin{pmatrix}
u \\ \lambda
\end{pmatrix}= \begin{pmatrix} f \\ 0 \end{pmatrix}$

In [12]:
ngs_sl =  NGSOperator(bnd_trace_matrix.T)  @ NGSOperator(sl.weak_form()) @  NGSOperator(bnd_trace_matrix) 
ngs_sl = emblam @ ngs_sl @ emblam.T   # 1,1 block

In [13]:
gfu=GridFunction(X)
print(a.mat.width)

1239


In [14]:
lhs=a.mat  + ngs_sl+ ngs_bnd1
print(lhs.height,f.vec.size,gfu.vec.size)

1239 1239 1239


In [15]:
ngs_sl.width

1239

In [16]:
solvers.GMRes(A=lhs, b=f.vec, pre=inv, tol=1e-6, printrates=True, x=gf.vec,freedofs=X.FreeDofs(),maxsteps=400)

Step 0, error =  8.618843847028359
Step 1 , error =  6.362159671514446
Step 2 , error =  1.2563188130094172
Step 3 , error =  1.1687771781664749
Step 4 , error =  0.3588774468958156
Step 5 , error =  0.33256888551373665
Step 6 , error =  0.1269071573561528
Step 7 , error =  0.10931064262003447
Step 8 , error =  0.051663089828412175
Step 9 , error =  0.042158296468342564
Step 10 , error =  0.024047731712517596
Step 11 , error =  0.021756614706139914
Step 12 , error =  0.01807370628456508
Step 13 , error =  0.013971885834106485
Step 14 , error =  0.01035582089397973
Step 15 , error =  0.007225566896702744
Step 16 , error =  0.005929541952040738
Step 17 , error =  0.004743081301146306
Step 18 , error =  0.0045061725441421134
Step 19 , error =  0.0032279438207703944
Step 20 , error =  0.0030286474283721153
Step 21 , error =  0.002111968528997098
Step 22 , error =  0.0019339938694159966
Step 23 , error =  0.001759395162544261
Step 24 , error =  0.0016741313135443955
Step 25 , error =  0.001

basevector

In [17]:
gfu, gflam= gf.components
Draw(gfu)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2007-270-gcfd30775a', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2…

In [18]:
vx = BaseVector(w)
vy = BaseVector(h)
vx[:] = 1
vy.FV().NumPy()[:] = op * vx

NameError: name 'w' is not defined

In [ ]:
print (vy)

In [ ]:
ngsop = NGSOperator(op)
x = ngsop.CreateRowVector()
y = ngsop.CreateColVector()
x[:] = 1
y.data = ngsop * x
print(y)

The product space can provide embedding matrices for the individual components:

$$
E_u = \left( \begin{array}{c} I \\ 0 \end{array} \right)
\qquad
E_\lambda = \left( \begin{array}{c} 0 \\ I \end{array} \right)
$$


In [ ]:
embu, emblam = X.embeddings

In [ ]:
ngs_dl = NGSOperator(dl.weak_form())
ngs_dl = emblam @ ngs_dl @ emblam.T   # 1,1 block

In [ ]:
len(ngs_dl.CreateRowVector())